In [1]:
import pandas as pd
import numpy as np

import plotly.express as px
from plotly.offline import *

In [30]:
# read in the data

repo = ('https://yipit-datatest.s3.us-east-2.amazonaws.com/Go.com_Assignment_Data_-3.csv')

godata = pd.read_csv(repo, skipinitialspace=True,usecols=['Date','Revenue','Product Line'], parse_dates=['Date'])

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

godata.head(5)

,Date,Revenue,Product Line
0,2015-01-01,"$108,632.40",Cars.go.com
1,2015-01-01,"$31,494.02",Planes.go.com
2,2015-01-01,"$239,376.48",Boats.go.com
3,2015-01-02,"$552,064.80",Cars.go.com
4,2015-01-02,"$238,344.12",Planes.go.com


In [59]:
# clean the data

godata['Revenue'] = [float(str(i).replace('$','').replace(',','').strip()) for i in godata.Revenue]
godata = godata.dropna()

0    108632.40
1     31494.02
2    239376.48
3    552064.80
4    238344.12
Name: Revenue, dtype: float64

In [64]:
# groupby and investigate

godata_group = godata.groupby(['Product Line']).agg({'Revenue':np.sum})

In [65]:
godata_group

,Revenue
Product Line,
Boats.go.com,4736347.68
Cars.go.com,16757524.80
Planes.go.com,32529742.79


## Data Irregularity ##

Note here that **Cars.go.com** had a blowout quarter: about 2.5 x as much revenue, when looking at the past four quarters combined. Should be confirmed with analysts.

Other domains (planes.go.com, boats.go.com) seem in line with expectations.

## Next Steps ##

As revenue data cannot be confirmed, the data is unreliable.

Must switch to profit data (linear model).

In [118]:
# create dataframe from custom formatted .xlsx document

profit = pd.DataFrame({
    'Product Line':['Cars','Planes','Boats'],
    'December 2013': [1217012.6, 161697.015,86591.56],
    'March 2014': [1132716.4, 174307.62, 88361.76],
    'June 2014': [1140398.8,180197.82,93143.68],
    'September 2014': [1138382, 172111.21, 91955.3],
    'December 2014': [1181270.4, 226688.21, 108744.28],
                      })

In [68]:
profit[:5]

,Product Line,December 2013,March 2014,June 2014,September 2014,December 2014
0,Cars,1217012.600,1132716.40,1140398.80,1138382.00,1181270.40
1,Planes,161697.015,174307.62,180197.82,172111.21,226688.21
2,Boats,86591.560,88361.76,93143.68,91955.30,108744.28


In [119]:
# melt to long form

profit_melt = pd.melt(profit, id_vars='Product Line', var_name='Quarter', value_name='Profit')

In [79]:
profit_melt

,Product Line,Quarter,Profit
0,Cars,December 2013,1217012.600
1,Planes,December 2013,161697.015
2,Boats,December 2013,86591.560
3,Cars,March 2014,1132716.400
4,Planes,March 2014,174307.620
5,Boats,March 2014,88361.760
6,Cars,June 2014,1140398.800
7,Planes,June 2014,180197.820
8,Boats,June 2014,93143.680
9,Cars,September 2014,1138382.000


In [92]:
# split the variables

xs = np.array([1,2,3,4,5]).reshape(-1,1)
y1 = profit_melt[profit_melt['Product Line'].isin(['Cars'])]
y2 = profit_melt[profit_melt['Product Line'].isin(['Planes'])]
y3 = profit_melt[profit_melt['Product Line'].isin(['Boats'])]

In [96]:
# train test split

x_train = xs
y1_train = y1[['Profit']].squeeze()
y2_train = y2[['Profit']].squeeze()
y3_train = y3[['Profit']].squeeze()

x_test = np.array([6]).reshape(-1,1)
# y_test = np.nan

In [105]:
# linear regression function

from sklearn.linear_model import LinearRegression


def MultipleLinearRegression(model, x_train, y_trains_lst, x_test, y_test_lst):

    clf = model

    predictions = []

    for y_train in y_trains_lst:

        clf.fit(x_train, y_train)
        predictions.append(clf.predict(x_test).round(2))

        print('the training score for this set is: ', clf.score(x_train, y_train))
        print('the test score for this set is unknown.')
        print('the RMSE for this set is unknown.')
        print('')    

    return predictions

In [106]:
# call the function

predictions = MultipleLinearRegression(LinearRegression(), x_train, [y1_train, y2_train, y3_train], x_test, np.nan)

the training score for this set is:  0.08205265989567812
the test score for this set is unknown.
the RMSE for this set is unknown.

the training score for this set is:  0.6367541611553372
the test score for this set is unknown.
the RMSE for this set is unknown.

the training score for this set is:  0.7432343947085194
the test score for this set is unknown.
the RMSE for this set is unknown.



## Conclusions ##

We receive three final floats, one for Cars, Planes and Boats.
These values are based on projects from just the past year. They are not considered to be accurate.

Our model does not use any training data besides the dependent variable. This could be improved with more data.

In [113]:
# look at predictions

domains_names = ['Cars','Planes','Boats']

predictions_dct = dict(zip(domains_names, predictions))

profit['March 2015'] = [int(i) for i in predictions]

In [114]:
# confirm and sum

profit[:5]

,Product Line,December 2013,March 2014,June 2014,September 2014,December 2014,March 2015
0,Cars,1217012.600,1132716.40,1140398.80,1138382.00,1181270.40,1142210
1,Planes,161697.015,174307.62,180197.82,172111.21,226688.21,221336
2,Boats,86591.560,88361.76,93143.68,91955.30,108744.28,108129


In [116]:
# final answer

Q12015_Profit = profit['March 2015'].sum()
Q12015_Profit

1471675

## Final Answer ##

The suggested Q1, 2015 Profit across Cars, Planes, and Boats domains is equal to 1471675.